In [11]:
from pymongo import MongoClient
from datetime import datetime

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["MCA_WT"]

customers = db['customers']
accounts = db['accounts']
customers.delete_many({})
accounts.delete_many({})
# Define sample customer documents
customers_data = [
    {"first_name": "Sarah", "last_name": "Smith", "date_opened": datetime(2020, 1, 1), "branch": "Main Branch"},
    {"first_name": "Sam", "last_name": "Brown", "date_opened": datetime(2021, 5, 10), "branch": "East Branch"},
    {"first_name": "John", "last_name": "Doe", "date_opened": datetime(2020, 1, 1), "branch": "Main Branch"},
    {"first_name": "Sandra", "last_name": "White", "date_opened": datetime(2019, 11, 25), "branch": "North Branch"},
    {"first_name": "Steve", "last_name": "Black", "date_opened": datetime(2020, 1, 1), "branch": "South Branch"}
]

# Insert customers and get their inserted IDs
inserted_customers = customers.insert_many(customers_data)
customer_ids = inserted_customers.inserted_ids  # List of inserted customer IDs

# Define sample accounts documents, linking each to a customer
accounts_data = [
    {"customer_id": customer_ids[0], "account_type": "Saving", "branch": "Main Branch", "balance": 5000.0},
    {"customer_id": customer_ids[1], "account_type": "Saving", "branch": "East Branch", "balance": 3000.0},
    {"customer_id": customer_ids[2], "account_type": "Checking", "branch": "Main Branch", "balance": 10000.0},
    {"customer_id": customer_ids[3], "account_type": "Loan", "branch": "North Branch", "balance": 7000.0},
    {"customer_id": customer_ids[4], "account_type": "Loan", "branch": "North Branch", "balance": 15000.0}
]

# Insert accounts
accounts.insert_many(accounts_data)


InsertManyResult([ObjectId('67207c3d3721bb95b442685b'), ObjectId('67207c3d3721bb95b442685c'), ObjectId('67207c3d3721bb95b442685d'), ObjectId('67207c3d3721bb95b442685e'), ObjectId('67207c3d3721bb95b442685f')], acknowledged=True)

In [12]:
# List names of all customers whose first name starts with "S"

# Query for customers whose first name starts with 'S'
for customer in customers.find({"first_name": {"$regex": "^S"}}):
    print(customer["first_name"], customer["last_name"])


# Special characters that have specific meanings:

# .: Matches any single character (except newline).
# ^: Matches the start of a string.
# $: Matches the end of a string.
# *: Matches 0 or more occurrences of the preceding element.
# +: Matches 1 or more occurrences of the preceding element.
# ?: Matches 0 or 1 occurrence of the preceding element.
# []: Matches any one of the characters inside the brackets.
# Example: [abc] matches "a", "b", or "c".
# |: Acts as a logical OR.
# Example: cat|dog matches "cat" or "dog".


# --> Quantifiers: Specify how many times an element can occur:

# {n}: Exactly n occurrences.
# {n,}: At least n occurrences.
# {n,m}: Between n and m occurrences.


# --> Escaping Metacharacters: To match a metacharacter literally, you need to escape it with a backslash \.

# Example: \. matches a period ..

# -->   Groups and Capturing:
# (...): Groups patterns together, allowing you to apply quantifiers to the whole group.
# Example: (abc)+ matches one or more occurrences of "abc".




Sarah Smith
Sam Brown
Sandra White
Steve Black


In [13]:
# List all customers who opened an account on 1/1/2020 in a specific branch

# Query for customers who opened an account on 1/1/2020 in a specific branch
branch_name = "Main Branch"  # specify the branch name
for customer in customers.find({"date_opened": datetime(2020, 1, 1), "branch": branch_name}):
    print(customer["first_name"], customer["last_name"])

Sarah Smith
John Doe


In [14]:
# List the names of customers where account_type is "Saving"

# Query for customers with "Saving" account type
saving_accounts = accounts.find({"account_type": "Saving"})
for account in saving_accounts:
    customer = customers.find_one({"_id": account["customer_id"]})
    print(customer["first_name"], customer["last_name"])

Sarah Smith
Sam Brown


In [15]:
# Count the total number of loan account holders in a specific branch

# Query to count loan account holders in a specific branch
branch_name = "North Branch"  # specify the branch name
loan_accounts_count = accounts.count_documents({"account_type": "Loan", "branch": branch_name})
print("Total loan account holders in", branch_name, ":", loan_accounts_count)

Total loan account holders in North Branch : 2
